In [87]:
from PySimpleAutomata import DFA, automata_IO
from functools import reduce
import json
import pdb
import re
import os
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/willian/.pyenv/versions/miniconda3-latest/envs/notebook/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['pylab']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [85]:
class FSM:
    def __init__(self, name, transitions, alphabet, states, initial_states, accepting_states, deterministic=False):
        self.name = name
        self.transitions = transitions
        self.alphabet = alphabet
        self.states = states
        self.initial_states = initial_states
        self.accepting_states = accepting_states
        self.deterministic = deterministic
        
    @classmethod
    def loadGenerator(cls, name, generator):
        transitions = generator.t.copy()
        alphabet = list(reduce(lambda acc, x: acc | set(x.keys()), list(generator.t.values()), set()))
        states = set()
        for s0, trans in generator.t.items():
            for x, s1s in trans.items():
                for s1 in s1s:
                    states |= {s1}
        states = sorted(states)
        initial_states = [states[0]]
        accepting_states = [generator.finalState]
        for state in states:
            if not state in list(transitions.keys()):
                transitions[state] = {}
        return cls(name, transitions, alphabet, states, initial_states, accepting_states)
    
    def determinize(self):
        transitions, states, accepting_states = nfa2dfa(self)
        t = {}
        n = 65
        alphabet = []
        stateNames = {}
        for state in sorted(set(states)):
            stateNames[state] = chr(n)
            n += 1
        states = [stateNames[x] for x in states]
        accepting_states = sorted(set([stateNames[x] for x in accepting_states]))
        for s0, x, s1 in transitions:
            alphabet.append(x)
            if not stateNames[s0] in t:
                t[stateNames[s0]] = {}
            t[stateNames[s0]][x]  = stateNames[s1]
        for s in states:
            if not s in list(t.keys()):
                t[s] = {}
        self.dfa = self.__class__(self.name, t, alphabet, states, chr(65), accepting_states, True)
        return self.dfa
    
    def image(self):
        folder = 'dfa' if self.deterministic else 'nfa'
        jsonFile = '{}/{}.json'.format(folder, self.name)
        trans = []
        replacements = {'': 'ε', '\n': '↵'}
        for s0, transition in self.transitions.items():
            for x, s1s in transition.items():
                x = replacements[x] if x in replacements else x
                if self.deterministic:
                    trans.append([s0, x, s1s])
                else:
                    for s1 in s1s:
                        trans.append([s0, x, s1])
        fsmJson = {
            "alphabet": self.alphabet,
            "states": self.states,
            "initial_state" if self.deterministic else "initial_states": self.initial_states,
            "accepting_states": self.accepting_states,
            "transitions": trans
        }
        with open(jsonFile, 'w') as f:
            json.dump(fsmJson, f)
        if self.deterministic:
            z = automata_IO.dfa_to_dot(automata_IO.dfa_json_importer(jsonFile), self.name, folder)
        else:
            z = automata_IO.nfa_to_dot(automata_IO.nfa_json_importer(jsonFile), self.name, folder)
        return z

In [80]:
def e_closure(state, transitions, visited=set()):
    result = {state}
    if '' in list(transitions[state].keys()):
        nextStates = transitions[state]['']
        for nextState in nextStates:
            if not nextState in visited:
                result |= {nextState} | e_closure(nextState, transitions, visited | {nextState})
    return result

def nfa2dfa(nfa):
    states = set()
    transitions = []
    finalStates = []
    searchStates = nfa.initial_states.copy()
    searchedStates = nfa.initial_states.copy()
    j = nfa.transitions.copy()
    while searchStates:
        currState = reduce(lambda acc, x: acc | e_closure(x, j), searchStates.pop(0), set())
        currString = ' '.join(sorted(currState))
        if reduce(lambda acc, x: acc or x in nfa.accepting_states, currState, False):
            finalStates.append(currString)
        trans = {}
        for item in currState:
            for symbol, s1s in j[item].items():
                if symbol != '':
                    if not symbol in trans:
                        trans[symbol] = set()
                    for s1 in s1s:
                        trans[symbol] |= e_closure(s1, j)
            for symbol, s1 in trans.items():
                s1String = ' '.join(sorted(s1))
                if not s1 in searchStates and not s1String in searchedStates:
                    searchStates.append(s1)
                transitions.append([currString, symbol, s1String])
                states |= {currString, s1String}
        searchedStates.append(currString)
    return transitions, list(states), finalStates

In [86]:
class MetaRecognizer:
    def __init__(self, rulesFile, fsmPath):   
        self.fsms = {}
        with open(rulesFile, "r") as f:
            grammar = f.read().split('\n\n')
        if os.path.exists(fsmPath):
            os.system("rm {}/* ".format(fsmPath))
        for line in grammar:
            lineExtract = re.search(r'(?P<num>^\d+)\) (?P<NT>\w+) = (?P<rule>.*) \.', line)
            rule = lineExtract.group('rule')
            num = lineExtract.group('num')
            nt = lineExtract.group('NT')
            fsm = FSMGenerator(list(rule), debug=False).run()
            self.fsms[nt] = FSM.loadGenerator(nt, fsm)
            with open(fsmPath+'{}-{}.json'.format(num, nt), 'w') as outfile:
                json.dump(fsm.t, outfile)
                
    def determinizeAll(self):
        for fsm in self.fsms.values():
            fsm.image()
            fsm.determinize()
            fsm.dfa.image()
                
    def getSymbols(self):
        t = set()
        for fsm in self.fsms.values():
            t |= set(fsm.dfa.alphabet)
        return list(t)

In [89]:
class FSMGenerator:
    def __init__(self, chars, Si='', Sf='1', debug=False):
        self.Si = Si
        self.Sf = Sf
        self.S = 0
        self.t = {}
        self.token = ''
        self.NT = []
        self.T = []
        self.running = True
        self.currentState = 0
        self.chars = list(chars)
        self.NTon = False
        self.Ton = False
        self.toJoin = [[0, None]]
        self.type = ''
        self.restartTree = False
        self.debug=debug
        
    def run(self):
        while self.running:
            char = self.nextChar()
            if char:
                self.readChar(char)
            else:
                self.finishToken()
                self.running = False
        self.toJoin[-1][1] = self.S
        self.S += 1
        for s0, s1 in self.toJoin:
            self.addTransition('0', '', s0)
            self.addTransition(s1, '', self.S)
        if self.type == '}':
            self.addTransition(self.S, '', 0)
            self.finalState = str(0)
        else:
            self.finalState = str(self.S)
            if self.type == ']':
                self.addTransition(0, '', self.S)
        return self
        
    def nextChar(self):
        return self.chars.pop(0) if len(self.chars) > 0 else False
    
    def insertToken(self, x):
        self.token += x
        
    def upState(self):
        self.S += 1
        if self.debug:
            print("state "+self.Si+str(self.S))
    
    def addTransition(self, Si, x, Sf):
        if x == "“\\n”": x = '\n'
        z = re.sub(r'[“”]', '', x)
        S = self.Si + str(Si)
        if not(S in self.t):
            self.t[S] = {}
        if not(x in self.t[S]):
            self.t[S][z] = []
        self.t[S][z].append(self.Si + str(Sf))
        if self.debug:
            print("Adding transition ({}) {}->{}".format(S, z, self.Si + str(Sf)))
            print("Now we have {}".format(self.t))
        
    def finishToken(self, T=True):
        if not(self.token): return
        if T and not self.token in ['letter', 'digit', 'special']: self.token = ':' + self.token
        self.addTransition(self.S, self.token, self.S+1)
        self.restartTree = False
        self.upState()
        if T:
            self.Ton = False
            self.T.append(self.token)
        else:
            self.NTon = False
            self.NT.append(self.token)
        self.token = ''
         
    def readChar(self, x):
        if self.debug:
            pdb.set_trace()
        if self.NTon:
            self.insertToken(x)
            if x == '”':
                self.finishToken(False)
        elif self.Ton:
            if x == ' ':
                self.finishToken(True)
            else:
                self.insertToken(x)
        elif x in ['{', '[', '(']: # Open recursion
            self.finishToken()
            #if len(self.toJoin) > 0: self.S += 1
            self.subFSM = FSMGenerator(self.chars, self.Si + str(self.S)+"-", self.Si + str(self.S+1)).run()
            self.NT += self.subFSM.NT
            self.T += self.subFSM.T
            if self.debug:
                print("\tgot tokens &{}& in {}.{} -> {}".format(self.subFSM.T + self.subFSM.NT, self.subFSM.S, self.subFSM.type, self.subFSM.t))
            self.addTransition(str(self.S), '', str(self.S)+"-0")
            self.addTransition(str(self.S) + '-' + str(self.subFSM.finalState), '', str(self.S+1))
            for item in self.subFSM.t:
                if item in self.t:
                    for leaf in self.subFSM.t[item]:
                        if not leaf in self.t[item]:
                                self.t[item][leaf] = []
                        for microstate in self.subFSM.t[item][leaf]:
                            if not microstate in self.t[item][leaf]:
                                self.t[item][leaf].append(microstate)
                else:
                    self.t[item] = self.subFSM.t[item]
            if self.debug:
                print("\t Current t: {}".format(self.t))
            self.upState()
            self.chars = self.subFSM.chars
        elif x in ['}', ']', ')']: # Close recursion
            self.type = x
            self.running = False
        elif x == ' ':
            pass
        elif x == '|':
            self.finishToken()
            self.toJoin[-1][1] = self.S
            self.upState()
            self.toJoin.append([self.S, None])
            self.restartTree = True
            if self.debug:
                print("Current toJoin: {}".format(self.toJoin))
        else:
            if x == '“':
                self.finishToken()
                self.NTon = True
            elif re.match(r'[a-zA-Z0-9]', x):
                self.finishToken()
                self.Ton = True
            self.insertToken(x)